This is one of the Objectiv [example notebooks](https://objectiv.io/docs/modeling/example-notebooks/). These notebooks can run [on your own data](https://objectiv.io/docs/modeling/get-started-in-your-notebook/), or you can instead run the [Demo](https://objectiv.io/docs/home/try-the-demo/) to quickly try them out.

# Explore your data

This example notebook shows how you can easily explore your data collected with Objectiv. [See here how to get started in your notebook](https://objectiv.io/docs/modeling/get-started-in-your-notebook/). 

We first have to instantiate the model hub and an Objectiv DataFrame object.

In [ ]:
# set the timeframe of the analysis
start_date = '2022-06-01'
end_date = None

In [ ]:
from modelhub import ModelHub
from bach import display_sql_as_markdown

# instantiate the model hub, set the default time aggregation to daily
# and get the application & path global contexts
modelhub = ModelHub(time_aggregation='%Y-%m-%d', global_contexts=['application', 'path'])
# get an Objectiv DataFrame within a defined timeframe
df = modelhub.get_objectiv_dataframe(start_date=start_date, end_date=end_date)

### Reference
* [modelhub.ModelHub.get_objectiv_dataframe](https://objectiv.io/docs/modeling/open-model-hub/api-reference/ModelHub/get_objectiv_dataframe/)

## A first look at the data

In [ ]:
# have a look at the event data, sorted by the user's session ID & hit
df.sort_values(['session_id', 'session_hit_number'], ascending=False).head()

### Reference
* [bach.DataFrame.sort_values](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/sort_values/)
* [bach.DataFrame.head](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/head/)

## Understanding the columns

In [ ]:
# see the data type for each column
df.dtypes

### Reference
* [bach.DataFrame.dtypes](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/dtypes/)

What’s in these columns:

* `day`: the day of the session as a date.
* `moment`: the exact moment of the event as a timestamp.
* `user_id`: the unique identifier of the user, based on the cookie.
* `location_stack`: a JSON-like column that stores information on the UI location that the event was triggered. [See the open taxonomy notebook](https://objectiv.io/docs/modeling/example-notebooks/open-taxonomy/) for more details.
* `event_type`: the type of event that is logged, e.g. a PressEvent.
* `stack_event_types`: the parents of the event_type as a hierarchical JSON structure.
* `session_id`: a unique incremented integer ID for each session. Starts at 1 for the selected data in the DataFrame.
* `session_hit_number`: an incremented integer ID for each hit in the session, ordered by moment.

Besides these 'standard' columns, additional columns that are extracted from the global contexts are in the DataFrame. In this case these columns are:
* `application`.
* `path`.

### Reference
For a more detailed understanding of Objectiv events in general, and especially the global contexts and location_stack data columns, see the open analytics taxonomy documentation:

* [Events](https://objectiv.io/docs/taxonomy/events).
* [Global contexts](https://objectiv.io/docs/taxonomy/global-contexts).
* [Location contexts](https://objectiv.io/docs/taxonomy/location-contexts).

The [open taxonomy notebook](open-taxonomy-how-to.ipynb#Location-stack-&-global-contexts) has several examples on how to use it in modeling.

## Your first Objectiv event data
Before we dig any deeper, let’s take a more global look at the data Objectiv is now tracking in your product. An easy way to do this, is by looking at it from the [‘root locations’](https://objectiv.io/docs/taxonomy/reference/location-contexts/RootLocationContext); the main sections in your product’s UI.

First, we want to extract data from the `location_stack` column and the global contexts 'application' and 'path' that contain all relevant context about the event for this example. [See the open taxonomy notebook](https://objectiv.io/docs/modeling/example-notebooks/open-taxonomy/) for more details.

In [ ]:
# add specific contexts to the data as columns
df['application_id'] = df.application.context.id
df['root_location'] = df.location_stack.ls.get_from_context_with_type_series(type='RootLocationContext', key='id')
df['path_id'] = df.path.context.id

In [ ]:
# now easily slice the data using the added context columns
event_data = modelhub.agg.unique_users(df, groupby=['application_id', 'root_location', 'path_id', 'event_type'])
event_data.sort_values(ascending=False).to_frame().head(20)

### Reference
* [using global context data](open-taxonomy-how-to.ipynb#Location-stack-&-global-contexts)
* [modelhub.SeriesLocationStack.ls](https://objectiv.io/docs/modeling/open-model-hub/api-reference/SeriesLocationStack/ls/)
* [modelhub.Aggregate.unique_users](https://objectiv.io/docs/modeling/open-model-hub/models/aggregation/unique_users/)
* [bach.DataFrame.sort_values](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/sort_values/)
* [bach.Series.to_frame](https://objectiv.io/docs/modeling/bach/api-reference/Series/to_frame/)

## Understanding product features

For every event, Objectiv captures where it occurred in your product’s UI, using a hierarchical stack of [LocationContexts](https://objectiv.io/docs/taxonomy/location-contexts). This means you can easily slice the data on any part of the UI that you’re interested in. [See the open taxonomy notebook](https://objectiv.io/docs/modeling/example-notebooks/open-taxonomy/) for more details. It also means you can make product features very readable and easy to understand; see below how.

In [ ]:
# add a readable product feature name to the dataframe as a column
df['feature_nice_name'] = df.location_stack.ls.nice_name

In [ ]:
# now easily look at the data by product feature
product_feature_data = modelhub.agg.unique_users(df, groupby=['feature_nice_name', 'event_type'])
product_feature_data.sort_values(ascending=False).to_frame().head(20)

### Reference
* [modelhub.SeriesLocationStack.ls](https://objectiv.io/docs/modeling/open-model-hub/api-reference/SeriesLocationStack/ls/)
* [modelhub.Aggregate.unique_users](https://objectiv.io/docs/modeling/open-model-hub/models/aggregation/unique_users/)
* [bach.DataFrame.sort_values](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/sort_values/)
* [bach.Series.to_frame](https://objectiv.io/docs/modeling/bach/api-reference/Series/to_frame/)
* [bach.DataFrame.head](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/head/)

## Get the SQL for any analysis
The SQL for any analysis can be exported with one command, so you can use models in production directly to simplify data debugging & delivery to BI tools like Metabase, dbt, etc. See how you can [quickly create BI dashboards with this](https://objectiv.io/docs/home/try-the-demo#creating-bi-dashboards).

In [ ]:
# show the underlying SQL for this dataframe - works for any dataframe/model in Objectiv
display_sql_as_markdown(product_feature_data)

## Where to go next
Now that you’ve had a first look at your new data collected with Objectiv, the best next step is to see the [basic product analytics example notebook](basic-product-analytics.ipynb). It shows you to how to easily get product analytics metrics straight from your raw Objectiv data.